# volVal trial structure generation, radd simulation, and experimental results

In [ ]:
#write simulator for various trial structures
#function for trial structure, simulate results, then experimental 

In [30]:
import matplotlib.pyplot as plt
import numpy as np
import random as rd
from matplotlib import animation, rc
from IPython.display import HTML 
import csv
%matplotlib inline
import pandas as pd

In [31]:
#function inputs for testing 
nTrials = 1000
lambdaV = 10
lambdaStop = 10  
ssd = np.array([200, 250, 300, 350, 450])

percStopTrials = .10
constantChangePoint = 1
muMin = .25
muMax = .75
sigmaMin = 0 


In [32]:
def genChangePoint(constantChangePoint, lambdaV, nTrials):

    #sample changePoint
    changePoint_vec = np.zeros((nTrials, 1))
    changePoints = []

    if constantChangePoint == 0:

        i = 1
        while np.sum(changePoints) < nTrials-1:
            changePoints += list(np.random.poisson(lambdaV,i))   

        #if the sum(epochs) > nTrials, remove the last changePoint 
        #if we want to impose fewer constraints to prioritize integrity of 
        #random selection, we can relax the absolute # of trials 
        if np.sum(changePoints) >= nTrials:
            changePoints = changePoints[0:-1]       

        #convert to trial number indices 
        changeIdx = list(np.cumsum(changePoints))
        #mark change point 
        changePoint_vec[changeIdx] = 1 

    elif constantChangePoint == 1:
        changePoints = np.int(np.random.poisson(lambdaV,1))
        changePoint_vec[changePoints::changePoints,:] = 1 
        changeIdx = list(np.cumsum(np.repeat(changePoints, np.sum(changePoint_vec))))

    #     print(changeIdx)
    #     print(np.sum(changePoint_vec)*changePoints, changePoints)
    
    return changePoint_vec

In [33]:
def genStop(percStopTrials, lambdaStop, nTrials):

    #stop trials 
    nStopTrials = np.int(percStopTrials * nTrials)
    stopPoints = np.random.poisson(lambdaStop,nStopTrials) #length
    stopIdx = np.cumsum(stopPoints) #indices

    while stopIdx[-1] >= nTrials: 
        stopPoints = np.random.poisson(lambdaStop,nStopTrials)
        stopIdx = np.cumsum(stopPoints)

    #assign delays to the stop indices 
    nSamplesPerDelay = np.int(nStopTrials/len(ssd))
    ssd_vec = np.zeros((nTrials,1))

    np.random.shuffle(stopIdx)

    #need to reshape, otherwise broadcasting problems
    ssd_repeatedVal = np.reshape(np.repeat(ssd, nSamplesPerDelay),(nStopTrials,1)) 
    ssd_vec[stopIdx] = ssd_repeatedVal
    
    return ssd_vec

In [34]:
def genMuSigma(muMin, muMax, sigmaMin, changePoint_vec, changeIdx, nTrials):


    #means and sigma
    nEpochs = np.int(np.sum(changePoint_vec)) + 1

    mu_p = np.random.uniform(muMin, muMax, np.int(nEpochs/2))
    mu_n = -1*mu_p
    muRewardDelta = np.hstack((mu_p, mu_n))
    np.random.shuffle(muRewardDelta)
    # print(muRewardDelta)

    #more control over separation between mean values of each epoch
    # muDiff = .05 #greater muDiff = less conflict BETWEEN epochs 
    # test=np.arange(muMin, muMax, muDiff)
    # muControl_shuffle = np.random.uniform(min(test), max(test), len(test))
    # print(test)

    sigmaMax = min(muMin,1-muMax) #needs to be chosen to ensure that r diff is !> 1 
    #choose smaller sigma (based on mu) to avoid violating boundary of 1 or 0 

    sigma = np.random.uniform(sigmaMin, sigmaMax, 1)


    #sample from a normal distribution to generate reward delta for each trial 
    rewardDelta = np.zeros((nTrials,1))

    #this adds 0 and the end idx to change idx
    chgidx = [0]+changeIdx+[nTrials-1]

    #constructs unique interval for each mu 
    intervals = [(chgidx[i],chgidx[i+1]) for i in range(len(chgidx)-1)]
    # print(intervals)

    #each iterator is linked to respective range 
    #zip iterates over two lists simultaneously  
    for interval,mu in zip(intervals,muRewardDelta):
        nElements = interval[1]-interval[0]
        rewardDelta[interval[0]:interval[1]]= np.reshape(np.random.normal(mu,sigma,nElements), 
                                                         (nElements,1))
    #the last element needs to be assigned because of the interval const. above 
    #(range is not inclusive)
    rewardDelta[-1] = np.random.normal(muRewardDelta[-1], sigma, 1)

    # print(rewardDelta)
    # print(muRewardDelta)
    
    return rewardDelta, muRewardDelta

In [35]:
def genBaseTargetReward(rewardDelta, nTrials):

    #reverse engineer the actual values for targets

    #assume baseline value of .5 for each
    rewardDelta_div = 0.5*rewardDelta 
    adjustedTargets = 0.5*np.ones((nTrials,2))

    #always add half and subtract half of target value
    adjustedTargets[:,0] +=  rewardDelta_div[:,0]
    adjustedTargets[:,1] -=  rewardDelta_div[:,0]
    
    t1_baseReward = adjustedTargets[:,0]
    t2_baseReward = adjustedTargets[:,1]


#     print(adjustedTargets)
    return t1_baseReward, t2_baseReward

    # plt.title("raw target reward probabilities")
    # plt.xlabel("trial")
    # plt.ylabel("reward probability")
    # plt.legend(["t1", "t2"])
    # plt.plot(adjustedTargets[:,0], 'b.')
    # plt.plot(adjustedTargets[:,1], 'g.')

    # plt.figure(figsize=(20, 20))
    # plt.title("difference between target reward probabilities")
    # plt.xlabel("trial")
    # plt.ylabel("$\Delta$ reward probability")
    # plt.legend(["t1", "t2"])
    # plt.plot(rewardDelta, 'g.')
    # plt.plot(rewardDelta, 'k')
    # plt.plot()


# animate trial structure

In [ ]:
# First set up the figure, the axis, and the plot element we want to animate
fig, ax = plt.subplots()

ax.set_xlim(( -2, 2))
ax.set_ylim((-2, 2))

sinegraph, = ax.plot([], [])
dot, = ax.plot([], [],'o', color='red')

x = np.linspace(0, 2, 1000)
y = np.cos(x)

In [ ]:
# initialization function: plot the background of each frame
def init():
    dot.set_data([], [])
    return (dot,)

In [ ]:
# animation function. This is called sequentially
def animate(i): 
#     y2 = np.ones(0, 2, 1000)
#     jitter = np.random.normal(0, 1, (100,1))
#     y=np.linspace(-1, 1, num=1000) * (x - 0.01 * i)
    sinegraph.set_data(x[:i],y[:i])
    dot.set_data(x[i],y[i])
    return (dot,)


In [ ]:
# call the animator. blit=True means only re-draw the parts that have changed.
anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=50, interval=10000, blit=True)

In [ ]:
HTML(anim.to_html5_video())

In [ ]:
def generateTrialStructure(nTrials, lambdaV, lambdaSS, ssd, sigmaMin, sigmaMax):
    changePoint_vec = genChangePoint(constantChangePoint, lambdaV, nTrials)#refactoring 
    ssd_vec = genStop(percStopTrials, lambdaStop, nTrials)
    rewardDelta, muRewardDelta = genMuSigma(muMin, muMax, sigmaMin, 
                                 changePoint_vec, changeIdx, nTrials)
    t1_baseReward, t2_baseReward = genBaseTargetReward(rewardDelta, nTrials)
    
    return t1_baseReward, t2_baseReward, ssd_vec, changePoint_vec, rewardDelta, 
            muRewardDelta, sigma

## print parameters to a csv file 

In [ ]:
taskParameters = [t1_baseReward, t2_baseReward, ssd_vec, changePoint_vec, rewardDelta, 
            muRewardDelta, sigma]
header = ("t1_baseReward, t2_baseReward, ssd_vec, changePointVec,
          "rewardDelta, muRewardDelta, sigma")
#was sigmaDelta, but held constant for this exp.         
np.savetxt('VolVal_TrialStructure.csv',file_mat, header = header, delimiter=',', comments = '')

In [ ]:
#call function 
#simulate 
#experimental data 